In [1]:
import time, os, threading, cv2
import numpy as np
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException

In [2]:
def scrape(path, headless=False):
    chrome_options = Options()
    if headless:
        chrome_options.add_argument("--headless")
    chrome_options.add_argument("--window-size=1920x1080")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--hide-scrollbars")
    
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=chrome_options)
    driver.get(path)
    driver.maximize_window()
    
    try:
        provided_username = "fourbrotherstrading@icloud.com"
        username_field = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, './/input[@name = "username"]')))
        username_field.send_keys(provided_username)
    except Exception as e:
        print(f"No email button found")
        
    # get the password id from website
    try:
        provided_password = "Muhssan7865"
        pass_field = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, './/input[@name = "password"]')))
        pass_field.send_keys(provided_password)
    except Exception as e:
        print(f"No password tab found")

    # get the login button name-value
    try:
        login_button = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, './/button[@id="login-button"]')))
        login_button.click()
    except Exception as e:
        print(f"No login button found")
    
    # cancel if any
    try:
        cancel = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, './/button[@title="Transfer"]'))) 
        if cancel:
            cancel.click()
    except Exception as e:
        print("No transfer")
    
    # accept button
    try:
        accept = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, './/button[@id="accept"]')))
        if accept:
            accept.click()
    except Exception as e:
        print("No accept")
        
    
    try:
        aucs =WebDriverWait(driver, 5).until(EC.presence_of_all_elements_located((By.XPATH, './/span[@class="toggle-switch_handle__3sAFS"]'))) 
        if aucs:
            aucs[6].click()
            time.sleep(2)
    except Exception as e:
        print("No auctions for this index number")
    
    time.sleep(5)
    return driver

# def record_website_screen(driver, output_file="live_rec.avi", fps=8):
#     try:
#         img = driver.get_screenshot_as_png()
#         nparr = np.frombuffer(img, np.uint8)
#         frame = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
#         height, width, _ = frame.shape
#         fourcc = cv2.VideoWriter_fourcc(*'XVID')
#         out = cv2.VideoWriter(output_file, fourcc, fps, (width, height))

#         frame_count = 0
#         while True:
#             img = driver.get_screenshot_as_png()
#             nparr = np.frombuffer(img, np.uint8)
#             frame = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
#             frame_count += 1
#             out.write(frame)
#             time.sleep(1/fps)
#     except Exception as e:
#         print(f"Error in screen recording: {e}")
#     finally:
#         out.release()
#         print(f"Recording saved, captured {frame_count} frames.")

# def start_recording(driver):
#     thread = threading.Thread(target=record_website_screen, args=(driver,))
#     thread.daemon = True
#     thread.start()
def save_full_screenshot(driver, prefix="bidding_status"):
    try:
        os.makedirs("screenshots", exist_ok=True)
        filename = f"screenshots/{prefix}.png"
        driver.save_screenshot(filename)
        print(f"📸 Screenshot saved: {filename}")
    except Exception as e:
        print(f"❌ Screenshot failed: {e}")
if __name__ == "__main__":
    path = "https://lanes.simulcast3.manheim.co.uk/buyer"
    driver = scrape(path, headless=False)
    print("Starting screen recording...")
    # start_recording(driver)
    
    print("Starting data extraction...")
    results = []
    try:
        registration = ''
        while True:
            try:
                if not driver.window_handles:
                    print("Browser window closed. Stopping script.")
                    break
                
                details = {}
                try:
                    # REG (Example: using a more general XPath - adjust as needed)
                    try:
                        reg = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, './/p[@class="auction-header_vehicle-reg__Q05d6"]'))) # Less specific
                        current_reg = reg.text.strip()
                        
                        if current_reg != registration:
                            registration = current_reg
                            details['Reg'] = current_reg
                    
                            try:
                                lot = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '(.//h1[@class="heading_dark-blue__CLomU heading_small__nirSh heading_heading-component__AFwdR "])[2]'))) # Less specific
                                details['lot'] = lot.text.strip()
                            except Exception as e:
                                print(f"Error getting lot:")

                            # Bidding History (Example: handling StaleElementReferenceException)
                            try:
                                data_card = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '(.//div[@class="bidding-history_timeline__1Cx4J"])[2]')))
                                prices_elements = WebDriverWait(data_card, 10).until(EC.presence_of_all_elements_located((By.XPATH, './/span[@class="amount"]')))
                                price = [p.text.strip() for p in prices_elements] # Extract all prices at once
                                details['Bidding History'] = price
                            except Exception as e:
                                print(f"Error getting prices")
                                
                            # bid text
                            try:
                                bid_text =WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '(.//li[contains(@class,"bidding-history_sold")  or contains(@class, "bidding-history_sold-provisional")])[1]'))) 
                                if bid_text:
                                    details['Bidding Status'] = bid_text.text.strip()
                                    if details['Bidding Status']:
                                        save_full_screenshot(driver, prefix=details['Reg'].replace(" ", "_"))
                            except StaleElementReferenceException:
                                print("Stale element, retrying...")
                                continue # Go to the next iteration of the loop
                            except Exception as e:
                                print("No text")
                        else:
                            print("no live data")
                    except Exception as e:
                        print(f"Error getting reg")
                        
                    time.sleep(3)
                    if details:
                        results.append(details)
                        df = pd.DataFrame(results, columns=["lot", "Reg", "Bidding History", "Bidding Status"])
                        # header = not os.path.exists("live_data.csv")
                        df.to_csv("live_data.csv", mode='a', header = not os.path.exists("live_data.csv"), index=False)
                except Exception as e:
                        print("No more cars")
            except KeyboardInterrupt:
                print("Stopped by user.")
                break    
            
    except KeyboardInterrupt:
        print("Stopped by user.")
    finally:
        driver.quit()
        print("Browser closed.")


Starting screen recording...
Starting data extraction...
Error getting reg
Error getting reg
Error getting reg
Error getting reg
Error getting reg
Error getting reg
Error getting reg
Error getting reg
Error getting reg
Error getting reg
Error getting reg
Error getting reg
Error getting reg
Error getting reg
Error getting reg
Error getting reg
Error getting reg
Error getting reg
Error getting reg
Error getting reg
Error getting reg
Error getting reg
Error getting reg
Browser closed.


InvalidSessionIdException: Message: invalid session id; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#invalidsessionidexception
Stacktrace:
	GetHandleVerifier [0x0x108ba83+63395]
	GetHandleVerifier [0x0x108bac4+63460]
	(No symbol) [0x0xed1f70]
	(No symbol) [0x0xf0daa8]
	(No symbol) [0x0xf3f266]
	(No symbol) [0x0xf3ae65]
	(No symbol) [0x0xf3a3e6]
	(No symbol) [0x0xea3a45]
	(No symbol) [0x0xea3f9e]
	(No symbol) [0x0xea442d]
	GetHandleVerifier [0x0x1308883+2672035]
	GetHandleVerifier [0x0x1303cba+2652634]
	GetHandleVerifier [0x0x10b2bca+223466]
	GetHandleVerifier [0x0x10a2cb8+158168]
	GetHandleVerifier [0x0x10a978d+185517]
	(No symbol) [0x0xea3710]
	(No symbol) [0x0xea2f1d]
	GetHandleVerifier [0x0x143db9c+3938492]
	BaseThreadInitThunk [0x0x766a7ba9+25]
	RtlInitializeExceptionChain [0x0x76f4c3ab+107]
	RtlClearBits [0x0x76f4c32f+191]


In [ ]:
df = pd.read_csv("live_data.csv")

df2 = df.drop_duplicates(subset=['Reg'], keep='first')

finalized_df = df2.to_csv("live_data_CLEANED.csv", index=False)
# delete the original df
os.remove("live_data.csv")

Second-to-last records saved successfully!


In [ ]:
import re
df1 = pd.read_csv("live_data_CLEANED.csv")
df2 = pd.read_csv("manheim_data_cleaned.csv")

merged_df = df2.merge(df1[['Reg',"Bidding History" , "Bidding Status"]], on='Reg', how='left')

# Save the updated dataset
output_path = "manheim_live_data_merged_final.csv"
merged_df.to_csv(output_path, index=False)

df = pd.read_csv("manheim_live_data_merged_final.csv")
df = df.sort_values(by='Make', ascending=True)

df['Bidding History'] = df['Bidding History'].apply(lambda x: [i.split(" ")[0].split("£")[-1].replace(",", "") for i in str(x).split() if "£" in i])
# df['Last Bid'] = df['Bidding History'].apply(lambda x : int(x[0]) if isinstance(x, list) and len(x) > 0 else "na")
df['Last Bid'] = df['Bidding History'].apply(
    lambda x: (
        int(re.sub(r'\D', '', x[-1]))  # take last and clean non-digits
        if isinstance(x, list) and len(x) > 0 and re.sub(r'\D', '', x[-1]) != ''
        else "na"
    )
)

df['Bidding Status'] = df['Bidding Status'].apply(lambda x: x.split(" ")[0] if isinstance(x, str) and " "in x else "na")
df.to_csv("Final_Manheim.csv", index=False)

# os.remove("live_data_CLEANED.csv")
# os.remove("manheim_data_cleaned.csv")
# os.remove("manheim_live_data_merged_final.csv")